In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import sys

if "google.colab" in sys.modules:
    !pip uninstall lightgbm -y
    !pip install lightgbm==3.3.1
    !pip install Levenshtein

import os
import gc
import time
import random
import pickle
import Levenshtein
import difflib
import multiprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm.auto import tqdm
from requests import get
from collections import Counter, defaultdict
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

Found existing installation: lightgbm 3.3.1
Uninstalling lightgbm-3.3.1:
  Successfully uninstalled lightgbm-3.3.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached lightgbm-3.3.1-py3-none-manylinux1_x86_64.whl (2.0 MB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def stratified_group_k_fold(X, y, groups, k, seed=None):
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices

In [ ]:
def get_distribution(y_vals):
    y_distr = Counter(y_vals)
    y_vals_sum = sum(y_distr.values())
    return [f'{y_distr[i] / y_vals_sum:.2%}' for i in range(np.max(y_vals) + 1)]

In [ ]:
## Parameters
class CFG:
    AUTHOR = "kuruton"
    expID = ""
    if "google.colab" in sys.modules:
        expID = get("http://172.28.0.2:9000/api/sessions").json()[0]["name"].split(".")[0].split("-")[0]
    ROOT_DIR = '/content/drive/MyDrive/Kaggle/Foursquare'
    DATASET_DIR = os.path.join(ROOT_DIR, 'Dataset')
    INPUT_DIR = os.path.join(ROOT_DIR, 'Input')
    OUTPUT_DIR = os.path.join(ROOT_DIR, 'Output')
    is_debug = False
    SEED = 2022
    num_neighbors = 20
    num_split = 5
    feat_columns = ['name', 'address', 'city', 
                'state', 'zip', 'categories']
    vec_columns = ['name', 'categories', 'address', 
                  'state']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(CFG.SEED)

In [ ]:
if not os.path.exists(os.path.join(CFG.OUTPUT_DIR, CFG.expID)):
    os.makedirs(os.path.join(CFG.OUTPUT_DIR, CFG.expID))

In [ ]:
%load_ext Cython

In [ ]:
%%cython
def LCS(str S, str T):
    cdef int i, j
    cdef list dp = [[0] * (len(T) + 1) for _ in range(len(S) + 1)]
    for i in range(len(S)):
        for j in range(len(T)):
            dp[i + 1][j + 1] = max(dp[i][j] + (S[i] == T[j]), dp[i + 1][j], dp[i][j + 1], dp[i + 1][j + 1])
    return dp[len(S)][len(T)]

In [ ]:
def recall_knn(df, Neighbors = 10):
    print('Start knn')
    train_df = []
    knn = NearestNeighbors(n_neighbors = Neighbors)
    knn.fit(df[['latitude','longitude']], df.index)
    dists, nears = knn.kneighbors(df[['latitude','longitude']])
    
    for k in range(Neighbors):            
        cur_df = df[['id']]
        cur_df['match_id'] = df['id'].values[nears[:, k]]
        cur_df['kdist'] = dists[:, k]
        cur_df['kneighbors'] = k
        train_df.append(cur_df)
    
    train_df = pd.concat(train_df)
    train_df = train_df.reset_index()
    
    return train_df

In [ ]:
def add_features(df, set_num):    
    for col in tqdm(CFG.feat_columns):       
        if col in CFG.vec_columns:
            tv_fit = tfidf_d[col]
            indexs = [id2index_d[i] for i in df['id']]
            match_indexs = [id2index_d[i] for i in df['match_id']]                    
            df[f'{col}_sim'] = tv_fit[indexs].multiply(tv_fit[match_indexs]).sum(axis = 1).A.ravel()
        
        col_values = data.loc[df['id']][col].values.astype(str)
        matcol_values = data.loc[df['match_id']][col].values.astype(str)
        vc = data.loc[data['set'] == set_num, col].value_counts() / sum(data['set'] == set_num)
        geshs = []
        levens = []
        jaros = []
        lcss = []
        vc1 = []
        vc2 = []

        for s, match_s in zip(col_values, matcol_values):
            if s != 'nan':
                vc1.append(vc[s])
            else:
                vc1.append(np.nan)

            if match_s != 'nan':
                vc2.append(vc[match_s])
            else:
                vc2.append(np.nan)

            if s != 'nan' and match_s != 'nan':                    
                geshs.append(difflib.SequenceMatcher(None, s, match_s).ratio())
                levens.append(Levenshtein.distance(s, match_s))
                jaros.append(Levenshtein.jaro_winkler(s, match_s))
                lcss.append(LCS(str(s), str(match_s)))
            else:
                geshs.append(np.nan)
                levens.append(np.nan)
                jaros.append(np.nan)
                lcss.append(np.nan)
        
        df[f'{col}_gesh'] = geshs
        df[f'{col}_leven'] = levens
        df[f'{col}_jaro'] = jaros
        df[f'{col}_lcs'] = lcss
        df[f'{col}_CE_1'] = vc1
        df[f'{col}_CE_2'] = vc2
        
        if col not in ['phone', 'zip']:
            df[f'{col}_len'] = list(map(len, col_values))
            df[f'match_{col}_len'] = list(map(len, matcol_values)) 
            df[f'{col}_len_diff'] = np.abs(df[f'{col}_len'] - df[f'match_{col}_len'])
            df[f'{col}_nleven'] = df[f'{col}_leven'] / \
                                    df[[f'{col}_len', f'match_{col}_len']].max(axis = 1)
            
            df[f'{col}_nlcsk'] = df[f'{col}_lcs'] / df[f'match_{col}_len']
            df[f'{col}_nlcs'] = df[f'{col}_lcs'] / df[f'{col}_len']
            
            df = df.drop(f'{col}_len', axis = 1)
            df = df.drop(f'match_{col}_len', axis = 1)
            gc.collect()
    return df

In [ ]:
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame):
    scores = []
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

def analysis(df):
    print('Num of data: %s' % len(df))
    print('Num of unique id: %s' % df['id'].nunique())
    print('Num of unique poi: %s' % df['point_of_interest'].nunique())
    
    poi_grouped = df.groupby('point_of_interest')['id'].count().reset_index()
    print('Mean num of unique poi: %s' % poi_grouped['id'].mean())

In [ ]:
## Data load
if "google.colab" in sys.modules:
    data_root = CFG.INPUT_DIR
else:
    data_root = '../input/foursquare-location-matching'
data = pd.read_csv(os.path.join(data_root, 'train.csv'))

if CFG.is_debug:
    data = data.sample(n = 10000, random_state = CFG.SEED)
    data = data.reset_index(drop = True)

In [ ]:
data['POI_count'] = data['point_of_interest'].map(dict(data.groupby('point_of_interest')['id'].apply(len)))
data.loc[data['POI_count'] > 5, 'POI_count'] = 5
data['POI_count'] -= 1
data.head()

id                     name   latitude   longitude  \
0  E_000001272c6c5d     Caf√© Stad Oudenaarde  50.859975    3.634196   
1  E_000002eae2a589           Carioca Manero -22.907225  -43.178244   
2  E_000007f24ebc95         ‡∏£‡πâ‡∏≤‡∏ô‡∏ï‡∏±‡∏î‡∏ú‡∏°‡∏Å‡∏≤‡∏£‡∏≤‡πÄ‡∏Å‡∏î  13.780813  100.484900   
3  E_000008a8ba4f48                 Turkcell  37.844510   27.844202   
4  E_00001d92066153  Restaurante Casa Cofi√±o  43.338196   -4.326821   

                  address        city            state   zip country  url  \
0             Abdijstraat  Nederename  Oost-Vlaanderen  9700      BE  NaN   
1                     NaN         NaN              NaN   NaN      BR  NaN   
2                     NaN         NaN              NaN   NaN      TH  NaN   
3  Adnan Menderes Bulvarƒ±         NaN              NaN   NaN      TR  NaN   
4                     NaN    Caviedes        Cantabria   NaN      ES  NaN   

  phone             categories point_of_interest  POI_count  
0   NaN                   Bars  P_677e840bb6fc7e          1  
1   NaN  Brazilian Restaurants  P_d82910d8382a83          1  
2   NaN   Salons / Barbershops  P_b1066599e78477          0  
3   NaN     Mobile Phone Shops  P_b2ed86905a4cd3          0  
4   NaN    Spanish Restaurants  P_809a884d4407fb          1

In [ ]:
distrs = [get_distribution(data['POI_count'])]
index = ['training set']

for set_ind, (dev_ind, val_ind) in enumerate(stratified_group_k_fold(data, data['POI_count'], data["point_of_interest"], k=2, seed=CFG.SEED)):
    data.loc[val_ind, "set"] = set_ind

    dev_y, val_y = data.loc[dev_ind, 'POI_count'], data.loc[val_ind, 'POI_count']
    dev_groups, val_groups = data.loc[dev_ind, "point_of_interest"], data.loc[val_ind, "point_of_interest"]
    
    assert len(set(dev_groups) & set(val_groups)) == 0
    
    distrs.append(get_distribution(dev_y))
    index.append(f'development set - set {set_ind}')
    distrs.append(get_distribution(val_y))
    index.append(f'validation set - set {set_ind}')

display('Distribution per class:')
pd.DataFrame(distrs, index=index, columns=[f'Label {l}' for l in range(np.max(data['POI_count']) + 1)])

'Distribution per class:'

Label 0 Label 1 Label 2 Label 3 Label 4
training set             37.32%  48.77%   6.38%   2.09%   5.44%
development set - set 0  37.32%  48.77%   6.38%   2.08%   5.44%
validation set - set 0   37.32%  48.77%   6.38%   2.09%   5.44%
development set - set 1  37.32%  48.77%   6.38%   2.09%   5.44%
validation set - set 1   37.32%  48.77%   6.38%   2.08%   5.44%

In [ ]:
# Data split
print('Num of train data: %s' % len(data))
print(data['set'].value_counts())

train_data = data.loc[data['set'] == 0]
test_data = data.loc[data['set'] == 1]

print('Train data: ')
analysis(train_data)
print('Test data: ')
analysis(test_data)

train_poi = train_data['point_of_interest'].unique().tolist()
test_poi = test_data['point_of_interest'].unique().tolist()

print(set(train_poi) & set(test_poi))

train_ids = train_data['id'].unique().tolist()
test_ids = test_data['id'].unique().tolist()
      
print(set(train_ids) & set(test_ids))
      
tv_ids_d = {}
tv_ids_d['train_ids'] = train_ids
tv_ids_d['test_ids'] = test_ids

np.save('tv_ids_d.npy', tv_ids_d)

del train_data, test_data
gc.collect()

data = data.set_index('id')
data = data.reset_index()

Num of train data: 1138812
0.0    569408
1.0    569404
Name: set, dtype: int64
Train data: 
Num of data: 569408
Num of unique id: 569408
Num of unique poi: 369986
Mean num of unique poi: 1.538998772926543
Test data: 
Num of data: 569404
Num of unique id: 569404
Num of unique poi: 369986
Mean num of unique poi: 1.538987961706659
set()
set()


In [ ]:
## Train data generated by knn
id2index_d = dict(zip(data['id'].values, data.index))

tfidf_d = {}
for col in CFG.vec_columns:
    tfidf = TfidfVectorizer()
    tfidf.fit(data[col].fillna('nan'))
    tv_fit = tfidf.transform(data[col].fillna('nan'))
    tfidf_d[col] = tv_fit

train_data = recall_knn(data.loc[data['set'] == 0], CFG.num_neighbors)
test_data = recall_knn(data.loc[data['set'] == 1], CFG.num_neighbors)

data = data.set_index('id')
ids = train_data['id'].tolist()
match_ids = train_data['match_id'].tolist()

poi = data.loc[ids]['point_of_interest'].values
match_poi = data.loc[match_ids]['point_of_interest'].values

train_data['label'] = np.array(poi == match_poi, dtype = np.int8)

test_ids = test_data['id'].tolist()
test_match_ids = test_data['match_id'].tolist()

test_poi = data.loc[test_ids]['point_of_interest'].values
test_match_poi = data.loc[test_match_ids]['point_of_interest'].values

test_data['label'] = np.array(test_poi == test_match_poi, dtype = np.int8)
del poi, match_poi, ids, match_ids, test_poi, test_match_poi, test_ids, test_match_ids
gc.collect()

print('Num of unique id: %s' % train_data['id'].nunique())
print('Num of train data: %s' % len(train_data))
print('Pos rate: %s' % train_data['label'].mean())
print(train_data.sample(5))

Start knn
Start knn
Num of unique id: 569408
Num of train data: 11388160
Pos rate: 0.08787117497471057
           index                id          match_id     kdist  kneighbors  \
3758063   683961  E_99a140a34f285e  E_c6d67cce880b89  0.004885           6   
4129113   286764  E_405f9c2c763e2a  E_3ff9414bca6b59  0.001146           7   
4251869   532418  E_77a7261e5e52fa  E_e9f5c8e7e65b69  0.003156           7   
10838572   39651  E_08e1f8e863777c  E_897b6d8bf8b758  0.009152          19   
348480    697719  E_9cb61b103c9671  E_9cb61b103c9671  0.000000           0   

          label  
3758063       0  
4129113       0  
4251869       0  
10838572      0  
348480        1  


In [ ]:
## Eval
data = data.reset_index()

id2poi = get_id2poi(data)
poi2ids = get_poi2ids(data)

eval_df = pd.DataFrame()
eval_df['id'] = data['id'].unique().tolist()
eval_df['match_id'] = eval_df['id']
print('Unique id: %s' % len(eval_df))

eval_df_ = train_data[train_data['label'] == 1][['id', 'match_id']]
eval_df = pd.concat([eval_df, eval_df_])

eval_df_ = test_data[test_data['label'] == 1][['id', 'match_id']]
eval_df = pd.concat([eval_df, eval_df_])

eval_df = eval_df.groupby('id')['match_id'].\
                        apply(list).reset_index()
eval_df['matches'] = eval_df['match_id'].apply(lambda x: ' '.join(set(x)))
print('Unique id: %s' % len(eval_df))

iou_score = get_score(eval_df)
print('IoU score: %s' % iou_score)

Unique id: 1138812
Unique id: 1138812
IoU score: 0.9202998613655506


In [ ]:
distrs = [get_distribution(train_data["label"])]
index = ['training set']

for fold_ind, (dev_ind, val_ind) in enumerate(stratified_group_k_fold(train_data, train_data["label"], train_data["id"], k=CFG.num_split, seed=CFG.SEED)):
    train_data.loc[val_ind, "fold"] = fold_ind

    dev_y, val_y = train_data.loc[dev_ind, "label"], train_data.loc[val_ind, "label"]
    dev_groups, val_groups = train_data.loc[dev_ind, "id"], train_data.loc[val_ind, "id"]
    
    assert len(set(dev_groups) & set(val_groups)) == 0
    
    distrs.append(get_distribution(dev_y))
    index.append(f'development set - fold {fold_ind}')
    distrs.append(get_distribution(val_y))
    index.append(f'validation set - fold {fold_ind}')

display('Distribution per class:')
pd.DataFrame(distrs, index=index, columns=[f'Label {l}' for l in range(np.max(train_data["label"]) + 1)])

'Distribution per class:'

Label 0 Label 1
training set              91.21%   8.79%
development set - fold 0  91.21%   8.79%
validation set - fold 0   91.21%   8.79%
development set - fold 1  91.21%   8.79%
validation set - fold 1   91.21%   8.79%
development set - fold 2  91.21%   8.79%
validation set - fold 2   91.21%   8.79%
development set - fold 3  91.21%   8.79%
validation set - fold 3   91.21%   8.79%
development set - fold 4  91.21%   8.79%
validation set - fold 4   91.21%   8.79%

# model learning

In [ ]:
data = data.set_index('id')
train_data = add_features(train_data, 0)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
features = ['kdist','kneighbors']

columns = ['name', 'address', 'city', 'state',
       'zip', 'categories']

for c in columns:
    if c == 'country':
        features += ['country_leven', 'country_sim', 'country_gesh', 'country_nleven']
        continue
    features += [f"{c}_gesh", f"{c}_jaro", f"{c}_lcs", f"{c}_leven"]
    if c == 'city':
        features += [f"{c}_len_diff", f"{c}_nleven", f"{c}_nlcsk", f"{c}_nlcs"]
    if c in ['address', 'categories', 'country', 'name', 'state', 'url']:
        features += [f"{c}_len_diff", f"{c}_nleven", f"{c}_nlcsk", f"{c}_nlcs", f"{c}_sim"]

for c in CFG.feat_columns:
    features += [c + "_CE_1", c + "_CE_2"]

print(len(features))
print(features)

62
['kdist', 'kneighbors', 'name_gesh', 'name_jaro', 'name_lcs', 'name_leven', 'name_len_diff', 'name_nleven', 'name_nlcsk', 'name_nlcs', 'name_sim', 'address_gesh', 'address_jaro', 'address_lcs', 'address_leven', 'address_len_diff', 'address_nleven', 'address_nlcsk', 'address_nlcs', 'address_sim', 'city_gesh', 'city_jaro', 'city_lcs', 'city_leven', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs', 'state_gesh', 'state_jaro', 'state_lcs', 'state_leven', 'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs', 'state_sim', 'zip_gesh', 'zip_jaro', 'zip_lcs', 'zip_leven', 'categories_gesh', 'categories_jaro', 'categories_lcs', 'categories_leven', 'categories_len_diff', 'categories_nleven', 'categories_nlcsk', 'categories_nlcs', 'categories_sim', 'name_CE_1', 'name_CE_2', 'address_CE_1', 'address_CE_2', 'city_CE_1', 'city_CE_2', 'state_CE_1', 'state_CE_2', 'zip_CE_1', 'zip_CE_2', 'categories_CE_1', 'categories_CE_2']


In [ ]:
def fit_lgb(X, y, params=None, es_rounds=20, seed=42,
             n_class=None, model_dir=None, folds=None):
    models = []

    trn_idx = folds!=0
    val_idx = folds==0
    X_train, y_train = X[trn_idx], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]

    if model_dir is None:
        model = lgb.LGBMClassifier(**params)
        model.fit(
            X_train, y_train, 
            eval_set=[(X_valid, y_valid)],  
            early_stopping_rounds=es_rounds, 
            eval_metric='binary',  
#             verbose=-1)
            verbose=50)
    else:
        with open(f'{model_dir}/lgb_fold{i}.pkl', 'rb') as f:
            model = pickle.load(f)
        
    oof = model.predict_proba(X_valid)
    models.append(model)
    
    file = os.path.join(CFG.OUTPUT_DIR, os.path.join(CFG.expID, f'lgb.pkl'))
    pickle.dump(model, open(file, 'wb'))
    print()

    cv = (oof.argmax(axis=-1) == y_valid).mean()
    print(f"CV-accuracy: {cv}")

    return oof, models

def inference_lgb(models, feat_df):
    pred = np.array([model.predict_proba(feat_df) for model in models])
    pred = np.mean(pred, axis=0)
    return pred

In [ ]:
params = {
    'objective': "binary",
    'learning_rate': 0.2,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': 42,

    'max_depth': 7,   
    'num_leaves': 35, 
    'n_estimators': 1000000, 
    "colsample_bytree": 0.9,
}


oof, models = fit_lgb(train_data[features], train_data["label"].astype(int), 
                      params=params, n_class=2, 
                      folds=train_data["fold"].values)

[50]	valid_0's binary_logloss: 0.0500899
[100]	valid_0's binary_logloss: 0.0472248
[150]	valid_0's binary_logloss: 0.0459493
[200]	valid_0's binary_logloss: 0.0450043
[250]	valid_0's binary_logloss: 0.0443908
[300]	valid_0's binary_logloss: 0.0439164
[350]	valid_0's binary_logloss: 0.0434102
[400]	valid_0's binary_logloss: 0.0429787
[450]	valid_0's binary_logloss: 0.0426591
[500]	valid_0's binary_logloss: 0.0423099
[550]	valid_0's binary_logloss: 0.0420533
[600]	valid_0's binary_logloss: 0.0417987
[650]	valid_0's binary_logloss: 0.0415369
[700]	valid_0's binary_logloss: 0.0412407
[750]	valid_0's binary_logloss: 0.041017
[800]	valid_0's binary_logloss: 0.0408101
[850]	valid_0's binary_logloss: 0.0406173
[900]	valid_0's binary_logloss: 0.0404615
[950]	valid_0's binary_logloss: 0.0402929
[1000]	valid_0's binary_logloss: 0.040128
[1050]	valid_0's binary_logloss: 0.039986
[1100]	valid_0's binary_logloss: 0.0398699
[1150]	valid_0's binary_logloss: 0.0397199
[1200]	valid_0's binary_logloss: 0

In [ ]:
del train_data
gc.collect()

85

#Check Feature Importances

In [ ]:
def plot_importances(models):
    importance_df = pd.DataFrame(models[0].feature_importances_, 
                                 index=features, 
                                 columns=['importance'])\
                        .sort_values("importance", ascending=False)

    plt.subplots(figsize=(len(features) // 4, 5))
    plt.bar(importance_df.index, importance_df.importance)
    plt.grid()
    plt.xticks(rotation=90)
    plt.ylabel("importance")
    plt.tight_layout()
    plt.show()

plot_importances(models)

#Check CV

In [ ]:
test_data = add_features(test_data, 1)
pred = inference_lgb(models, test_data[features])
test_data['pred'] = pred[:, 1]

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
def post_process(df):
    id2match = dict(zip(df['id'].values, df['matches'].str.split()))

    for base, match in df[['id', 'matches']].values:
        match = match.split()
        if len(match) == 1:        
            continue

        for m in match:
            if base not in id2match[m]:
                id2match[m].append(base)
    df['matches'] = df['id'].map(id2match).map(' '.join)
    return df 

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_uniform('threshold', 0, 1)
    test_pred_df = test_data[test_data['pred'] > x][['id', 'match_id']]
    out_df = pd.DataFrame()
    out_df['id'] = test_data['id'].unique().tolist()
    out_df['match_id'] = out_df['id']
    out_df = pd.concat([out_df, test_pred_df])
    out_df = out_df.groupby('id')['match_id'].\
                        apply(list).reset_index()
    out_df['matches'] = out_df['match_id'].apply(lambda x: ' '.join(set(x)))
    out_df = post_process(out_df)

    score = get_score(out_df)
    print(f"CV: {score:.6f}")
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2022-06-02 05:51:40,300] A new study created in memory with name: no-name-95060b41-11b0-44dd-91b5-1c7d59003828
[I 2022-06-02 05:52:06,317] Trial 0 finished with value: 0.8165655754800571 and parameters: {'threshold': 0.887285463439302}. Best is trial 0 with value: 0.8165655754800571.


CV: 0.816566


[I 2022-06-02 05:52:23,135] Trial 1 finished with value: 0.6952471909470158 and parameters: {'threshold': 0.9990510824604507}. Best is trial 0 with value: 0.8165655754800571.


CV: 0.695247


[I 2022-06-02 05:52:42,190] Trial 2 finished with value: 0.8225965195315761 and parameters: {'threshold': 0.8482264624662945}. Best is trial 2 with value: 0.8225965195315761.


...

CV: 0.834529


[I 2022-06-02 06:23:23,074] Trial 98 finished with value: 0.8337948569094809 and parameters: {'threshold': 0.6574162961758254}. Best is trial 53 with value: 0.8345822124008468.


CV: 0.833795


[I 2022-06-02 06:23:42,439] Trial 99 finished with value: 0.8339988033612611 and parameters: {'threshold': 0.5200174683726168}. Best is trial 53 with value: 0.8345822124008468.


CV: 0.833999


In [ ]:
threshold = study.best_params['threshold']
threshold

0.5639626284788222

In [ ]:
study.best_value

0.8345822124008468

In [ ]:
test_pred_df = test_data[test_data['pred'] > threshold][['id', 'match_id']]
out_df = pd.DataFrame()
out_df['id'] = test_data['id'].unique().tolist()
out_df['match_id'] = out_df['id']
out_df = pd.concat([out_df, test_pred_df])
out_df = out_df.groupby('id')['match_id'].\
                    apply(list).reset_index()
out_df['matches'] = out_df['match_id'].apply(lambda x: ' '.join(set(x)))
out_df = post_process(out_df)

score = get_score(out_df)
print(f"CV: {score:.6f}")

CV: 0.834582


# ÈñìÈÅï„Å£„Å¶„ÅÑ„Çã„ÇÇ„ÅÆ„ÅÆ‰∏≠Ë∫´„ÇíË¶ã„Çã

In [ ]:
data = data.reset_index()

label„Åå1„Å™„ÅÆ„Å´0„Å®‰∫àÊ∏¨„Åó„Å¶„ÅÑ„Çã„ÇÇ„ÅÆ

In [ ]:
false_negative = test_data.loc[(test_data["label"] == 1) & (test_data["pred"] <= threshold), :]
false_negative.describe()

index          kdist     kneighbors     label       name_sim  \
count  1.358390e+05  135839.000000  135839.000000  135839.0  135839.000000   
mean   5.704782e+05       0.007891       6.294061       1.0       0.262020   
std    3.291375e+05       0.071397       5.254262       0.0       0.297326   
min    7.000000e+00       0.000000       0.000000       1.0       0.000000   
25%    2.864000e+05       0.000450       2.000000       1.0       0.000000   
50%    5.677360e+05       0.001116       5.000000       1.0       0.164300   
75%    8.570690e+05       0.003052      10.000000       1.0       0.488334   
max    1.138810e+06       4.448372      19.000000       1.0       1.000000   

           name_gesh     name_leven      name_jaro       name_lcs  \
count  135839.000000  135839.000000  135839.000000  135839.000000   
mean        0.404821      16.708007       0.571189       8.450438   
std         0.247940       9.434542       0.226265       6.377583   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.205128      10.000000       0.447619       4.000000   
50%         0.387097      16.000000       0.561610       7.000000   
75%         0.582278      22.000000       0.698413      12.000000   
max         1.000000      59.000000       1.000000      53.000000   

           name_CE_1  ...  categories_leven  categories_jaro  categories_lcs  \
count  135839.000000  ...     101908.000000    101908.000000   101908.000000   
mean        0.000028  ...         12.447776         0.674120        8.914119   
std         0.000283  ...         11.525406         0.242862        7.190790   
min         0.000002  ...          0.000000         0.000000        0.000000   
25%         0.000002  ...          0.000000         0.483041        4.000000   
50%         0.000002  ...         12.000000         0.584046        7.000000   
75%         0.000002  ...         17.000000         1.000000       12.000000   
max         0.004961  ...         87.000000         1.000000       77.000000   

       categories_CE_1  categories_CE_2  categories_len_diff  \
count    115694.000000    116196.000000        135839.000000   
mean          0.004897         0.004862             9.045848   
std           0.005908         0.005880            10.911218   
min           0.000002         0.000002             0.000000   
25%           0.000715         0.000706             0.000000   
50%           0.002529         0.002503             6.000000   
75%           0.006027         0.005943            13.000000   
max           0.022032         0.022032            91.000000   

       categories_nleven  categories_nlcsk  categories_nlcs          pred  
count      101908.000000     101908.000000    101908.000000  1.358390e+05  
mean            0.538514          0.578882         0.581443  1.844631e-01  
std             0.352432          0.329251         0.329448  1.752039e-01  
min             0.000000          0.000000         0.000000  4.673805e-10  
25%             0.000000          0.285714         0.285714  2.449011e-02  
50%             0.714286          0.470588         0.473684  1.262738e-01  
75%             0.823529          1.000000         1.000000  3.238607e-01  
max             1.000000          1.000000         1.000000  5.639623e-01  

[8 rows x 65 columns]

In [ ]:
false_negative.sample(min(false_negative.shape[0], 30))

index                id          match_id     kdist  kneighbors  \
2790209   1024916  E_e648e7aed48edd  E_82734c8e1c10d6  0.003437           4   
1074906   1010804  E_e3113b1f3f918d  E_7db4c43deb6c08  0.000337           1   
1976735    536647  E_7898405513ace1  E_fb6290eb6d5c5d  0.000494           3   
5068579   1026372  E_e69cb833ca9396  E_9de6b67fa4710c  0.000437           8   
778696     418386  E_5dfe0057fe3fac  E_af38084bf295ba  0.038953           1   
1015389    891699  E_c84eec4d5c1378  E_fc9d593151bdc4  0.000261           1   
1224847    172001  E_2691fae53ad808  E_057a9238e67ac3  0.000688           2   
2850373      6720  E_0184dd91b7fbdd  E_79a1ef57468b10  0.001172           5   
2857770     21472  E_04d37344e3b493  E_a0b65b50fe2d13  0.000404           5   
1814374    212093  E_2f9f5a5d1759c5  E_518adf9470b87c  0.002488           3   
698862     258567  E_3a0d8b359567f1  E_7d6fe886d5f9ba  0.002488           1   
1445525    612555  E_89b848be990c74  E_9c54177ba37e9a  0.000941           2   
4530647   1089509  E_f4d3a0db7a578a  E_b2ab5d498163e9  0.001875           7   
6166301    944238  E_d41aa363358d3d  E_21018ee92a9d4f  0.010161          10   
7081733    497122  E_6fbfab22d5f242  E_0eb6a659b24128  0.003298          12   
4858756    606213  E_883e96a7517a7b  E_541a5fac7e3495  0.001048           8   
772482     405797  E_5b31ce7bdaf7c4  E_9c228340476feb  0.000445           1   
11307358   977277  E_db8b5fc51924ef  E_a54b29413c4178  0.000237          19   
1086081   1033062  E_e81d42fea68920  E_33c47d775ff2e4  0.000148           1   
7579167    353430  E_4f6b7c71786206  E_fc433ed29cf86a  0.001472          13   
5929216    469679  E_698e3e2eaa27b9  E_545718b5fa2203  0.001274          10   
4314579    656741  E_939407332d1d74  E_778696f4414f85  0.000677           7   
893741     647848  E_919a34535d7dfe  E_26b99f2b53ab03  0.000687           1   
7477469    150544  E_21bf869fc0c0e6  E_8ace4a0bc5515f  0.000444          13   
1436883    595278  E_85cbe67d5becdd  E_ae4b228e29f23c  0.000228           2   
1443666    608854  E_88dde0b8b695f1  E_fb41e30a3f3b81  0.000670           2   
5724144     60316  E_0d7eb188d97b4d  E_312a57e3605bab  0.001168          10   
9218111    214973  E_3041b427cb0dcf  E_63fb661d4e999b  0.002963          16   
9589703    958296  E_d746310c2a7884  E_666e5251399f6b  0.005327          16   
6303300     79778  E_11d639aa25f53d  E_ed738a8c9d7af3  0.004796          11   

          label  name_sim  name_gesh  name_leven  name_jaro  ...  \
2790209       1  0.070263   0.291667        43.0   0.528101  ...   
1074906       1  0.000000   0.285714        34.0   0.559981  ...   
1976735       1  0.307100   0.464286        26.0   0.584644  ...   
5068579       1  0.000000   0.000000        22.0   0.000000  ...   
778696        1  0.000000   0.000000        14.0   0.000000  ...   
1015389       1  0.000000   0.200000         6.0   0.492063  ...   
1224847       1  0.396663   0.790698         9.0   0.835080  ...   
2850373       1  0.000000   0.666667         7.0   0.867492  ...   
2857770       1  0.000000   0.140351        28.0   0.547908  ...   
1814374       1  0.000000   0.160000        23.0   0.612027  ...   
698862        1  0.000000   0.226415        24.0   0.453065  ...   
1445525       1  0.456465   0.588235        22.0   0.850948  ...   
4530647       1  0.500577   0.491803        28.0   0.598953  ...   
6166301       1  0.435058   0.580645        13.0   0.881818  ...   
7081733       1  0.148636   0.478873        23.0   0.668049  ...   
4858756       1  0.000000   0.000000         5.0   0.000000  ...   
772482        1  0.000000   0.357143        13.0   0.539852  ...   
11307358      1  0.000000   0.291667        18.0   0.515520  ...   
1086081       1  0.000000   0.000000        17.0   0.000000  ...   
7579167       1  0.000000   0.000000         8.0   0.000000  ...   
5929216       1  0.233156   0.604651        10.0   0.634088  ...   
4314579       1  0.000000   0.210526        18.0   0.473684  ...  

In [ ]:
for id, match_id in false_negative.sample(min(false_negative.shape[0], 30))[['id', 'match_id']].values:
    display(data[data['id'].isin([id, match_id])])

id                                              name  \
594655  E_85a798e1e07c52  Pathumwan Princess Hotel (‡πÇ‡∏£‡∏á‡πÅ‡∏£‡∏°‡∏õ‡∏ó‡∏∏‡∏°‡∏ß‡∏±‡∏ô‡∏õ‡∏£‡∏¥‡πä‡∏ô‡πÄ‡∏ã‡∏™)   
976784  E_db6de9537688e1                       Pratumwan Princess's Palace   

         latitude   longitude            address        city    state    zip  \
594655  13.743255  100.529905  444 Phaya Thai Rd  Pathum Wan  Bangkok  10330   
976784  13.743518  100.530290                NaN         NaN      NaN    NaN   

       country                       url     phone categories  \
594655      TH  http://www.pprincess.com  22163700     Hotels   
976784      TH                       NaN       NaN        NaN   

       point_of_interest  POI_count  set  
594655  P_26b49f2cefaa63          2  1.0  
976784  P_26b49f2cefaa63          2  1.0

id                                  name  latitude  \
431604  E_61015f39c55f49            NOVOTEL HOTEL NU AING!!!!!  -6.90491   
893782  E_c8c5685be77432  Fron Desk & GRO Desk Novotel Bandung  -6.90487   

         longitude              address     city      state    zip country  \
431604  107.604357  Jl Cihampelas 23-25  Bandung      jabar  40000      ID   
893782  107.603500           Cihampelas  Bandung  Indonesia  40241      ID   

        url phone categories point_of_interest  POI_count  set  
431604  NaN   NaN     Hotels  P_82252ce187cee8          4  1.0  
893782  NaN   NaN    Offices  P_82252ce187cee8          4  1.0

id                 name   latitude  longitude  \
157148  E_233506062e0b35  Teras Caf√© NataVega  39.888171  32.933347   
613780  E_89fc91b8d9567a      Vadi Teras Cafe  39.887482  32.934960   

              address    city   state  zip country  url phone categories  \
157148            NaN     NaN     NaN  NaN      TR  NaN   NaN      Caf√©s   
613780  nata vega avm  Ankara  Ankara  NaN      TR  NaN   NaN      Caf√©s   

       point_of_interest  POI_count  set  
157148  P_2444081b502290          1  1.0  
613780  P_2444081b502290          1  1.0

id                 name   latitude  longitude address  \
128021  E_1cb20c95cc7f48             Vodatech  40.657978  29.271134     NaN   
225227  E_32916661b49bbf  Vodatech Callcenter  40.657944  29.270714     NaN   

       city state  zip country  url phone           categories  \
128021  NaN   NaN  NaN      TR  NaN   NaN            Buildings   
225227  NaN   NaN  NaN      TR  NaN   NaN  Airport Food Courts   

       point_of_interest  POI_count  set  
128021  P_13a49b82bb391c          3  1.0  
225227  P_13a49b82bb391c          3  1.0

id                                   name  latitude  \
447227  E_64827aa1473e84                 Wajan Oriental Cuisine -6.957881   
710201  E_9f857c429a5471  Wajan noodle oriental culinair&coffee -6.959700   

         longitude                        address     city      state  zip  \
447227  107.561428  Taman Kopo Indah II, Ruko A 1  Bandung  West Java  NaN   
710201  107.561330            Taman Kopo Indah II      NaN        NaN  NaN   

       country  url phone           categories point_of_interest  POI_count  \
447227      ID  NaN   NaN  Chinese Restaurants  P_35b922462c7c03          3   
710201      ID  NaN   NaN  Chinese Restaurants  P_35b922462c7c03          3   

        set  
447227  1.0  
710201  1.0

...

id                       name  latitude   longitude  \
195316  E_2bd28d6a5b8423  PandanwangiPark Residence -7.951360  112.656998   
633856  E_8e82d8e0172318     Pandanwangi Royal Park -7.956911  112.659058   

                      address    city      state    zip country  url phone  \
195316                    NaN     NaN        NaN    NaN      ID  NaN   NaN   
633856  Jl. Simp. L.A Sucipto  Malang  East Java  65124      ID  NaN   NaN   

                  categories point_of_interest  POI_count  set  
195316  Housing Developments  P_f50f679f1f72b5          3  1.0  
633856  Housing Developments  P_f50f679f1f72b5          3  1.0

id       name   latitude  longitude         address  \
82917   E_128e39e59cb703     –õ–∏—Ç-—Ä–∞  44.963238  34.114240             NaN   
590715  E_84c725f04d148d  –¢–æ–ø –ö–ª–∞—Å—Å  44.962865  34.114587  –õ–µ—Ä–º–æ–Ω—Ç–æ–≤–∞ 14–∞   

               city state  zip country  url phone      categories  \
82917           NaN   NaN  NaN      UA  NaN   NaN  Middle Schools   
590715  –°–∏–º—Ñ–µ—Ä–æ–ø–æ–ª—å   NaN  NaN      UA  NaN   NaN             NaN   

       point_of_interest  POI_count  set  
82917   P_65769c7e8a4d68          4  1.0  
590715  P_65769c7e8a4d68          4  1.0

id                           name   latitude  longitude  \
41932   E_0960cffcfd84e2  Ankara 8. Aƒüƒ±r Ceza Mahkemesi  39.930567  32.851138   
373921  E_54043266d0be2a   Ankara 7. T√ºketici Mahkemesi  39.930614  32.852688   

       address city state  zip country  url phone  \
41932      NaN  NaN   NaN  NaN      TR  NaN   NaN   
373921     NaN  NaN   NaN  NaN      TR  NaN   NaN   

                              categories point_of_interest  POI_count  set  
41932   Courthouses, Prisons, Pet Stores  P_e00b9832620943          4  1.0  
373921              Government Buildings  P_e00b9832620943          4  1.0

id     name   latitude  longitude           address  \
42017   E_0964a34140f0bb  Jarrold  52.629466   1.293761    1-11 London St   
332883  E_4ad3ceb01b493f  JARROLD  52.628991   1.296392  5 BEDFORD STREET   

           city    state      zip country                        url  \
42017   Norwich  Norfolk  NR2 1JF      GB  https://www.jarrold.co.uk   
332883  NORWICH  Norfolk  NR2 1AL      GB                        NaN   

              phone               categories point_of_interest  POI_count  set  
42017   01603660661        Department Stores  P_60e5c815e54e7a          2  1.0  
332883   1603697107  Furniture / Home Stores  P_60e5c815e54e7a          2  1.0

id                                               name  \
388139  E_572e3a50679ffb    AFYONKARAHƒ∞SAR MESLEKƒ∞ VE TEKNƒ∞K ANADOLU Lƒ∞SESƒ∞   
694183  E_9bec16f28d95e0  Afyonkarahisar End√ºstri Meslek Lisesi Elektiri...   

         latitude  longitude         address   city state  zip country  url  \
388139  38.751205  30.544159    Kurtulu≈ü Cad  Afyon   NaN  NaN      TR  NaN   
694183  38.751056  30.544019  Afyonkarahisar    NaN   NaN  NaN      TR  NaN   

       phone    categories point_of_interest  POI_count  set  
388139   NaN           NaN  P_8b7ed7c7c057a2          2  1.0  
694183   NaN  High Schools  P_8b7ed7c7c057a2          2  1.0

label„Åå0„Å™„ÅÆ„Å´1„Å®‰∫àÊ∏¨„Åó„Å¶„ÅÑ„Çã„ÇÇ„ÅÆ

In [ ]:
false_positive = test_data.loc[(test_data["label"] == 0) & (test_data["pred"] > threshold), :]
false_positive.describe()

index         kdist    kneighbors    label      name_sim  \
count  5.311500e+04  53115.000000  53115.000000  53115.0  53115.000000   
mean   5.664274e+05      0.004671      4.633192      0.0      0.486672   
std    3.291557e+05      0.074910      4.830365      0.0      0.346155   
min    1.000000e+01      0.000000      0.000000      0.0      0.000000   
25%    2.782625e+05      0.000294      1.000000      0.0      0.146107   
50%    5.646230e+05      0.000821      2.000000      0.0      0.528946   
75%    8.529290e+05      0.002466      7.000000      0.0      0.765737   
max    1.138811e+06     11.519732     19.000000      0.0      1.000000   

          name_gesh    name_leven     name_jaro      name_lcs     name_CE_1  \
count  53091.000000  53091.000000  53091.000000  53091.000000  53096.000000   
mean       0.576258     13.055471      0.710982     11.407075      0.000055   
std        0.267037      9.686986      0.229618      7.370997      0.000389   
min        0.000000      0.000000      0.000000      0.000000      0.000002   
25%        0.380952      6.000000      0.550890      6.000000      0.000002   
50%        0.600000     12.000000      0.743939     10.000000      0.000002   
75%        0.784314     19.000000      0.900000     15.000000      0.000002   
max        1.000000     57.000000      1.000000     57.000000      0.004961   

       ...  categories_leven  categories_jaro  categories_lcs  \
count  ...      40671.000000     40671.000000    40671.000000   
mean   ...         10.019326         0.784788       11.401588   
std    ...         12.204240         0.239826        7.862276   
min    ...          0.000000         0.000000        0.000000   
25%    ...          0.000000         0.540341        5.000000   
50%    ...          7.000000         0.889411       10.000000   
75%    ...         16.000000         1.000000       15.000000   
max    ...         82.000000         1.000000       93.000000   

       categories_CE_1  categories_CE_2  categories_len_diff  \
count     46328.000000     45128.000000         53115.000000   
mean          0.004768         0.004742             8.748320   
std           0.005754         0.005748            11.647442   
min           0.000002         0.000002             0.000000   
25%           0.000525         0.000495             0.000000   
50%           0.002460         0.002460             4.000000   
75%           0.006361         0.006361            14.000000   
max           0.022032         0.022032            90.000000   

       categories_nleven  categories_nlcsk  categories_nlcs          pred  
count       40671.000000      40671.000000     40671.000000  53115.000000  
mean            0.374833          0.737116         0.737416      0.790083  
std             0.364259          0.323520         0.324296      0.134194  
min             0.000000          0.000000         0.000000      0.563965  
25%             0.000000          0.416667         0.416667      0.670546  
50%             0.416667          1.000000         1.000000      0.789190  
75%             0.750000          1.000000         1.000000      0.913136  
max             1.000000          1.000000         1.000000      0.999998  

[8 rows x 65 columns]

In [ ]:
false_positive.sample(min(false_positive.shape[0], 30))

index                id          match_id     kdist  kneighbors  \
1095162   1051251  E_ec3549e01576c2  E_fc7c7dcc82324e  0.000211           1   
4395281    818279  E_b7c3c19353b5cc  E_f35801d6eeddde  0.000334           7   
570468       2102  E_007bff40c64807  E_b54f9de866dffa  0.001596           1   
739550     339891  E_4c66a90c0fa8b1  E_d10208639dd1f6  0.000160           1   
4838301    565758  E_7f27d7439202ae  E_9df12191214dc2  0.217750           8   
1234786    191587  E_2afce65dbdc9f7  E_bfd769e5a8485c  0.000064           2   
3894120    955128  E_d6915cf3611c2a  E_08b7818c2093df  0.001663           6   
2033713    650217  E_921b1b5001134f  E_205caa72788292  0.011155           3   
937628     735835  E_a54d72d4e99522  E_11b8a9f087de1a  0.000225           1   
955102     770737  E_ad1d14de8a825b  E_31933fe883e5fd  0.000185           1   
3056303    418371  E_5dfd02e43f0b33  E_4bb433c7ec42d2  0.000253           5   
7505892    206934  E_2e6f275611f22b  E_c8417b305f36ff  0.003776          13   
1272298    266573  E_3bdf8344eaae3d  E_8367d2be3151eb  0.000758           2   
985705     832090  E_bad9979a34ec5b  E_6038c04e195fc2  0.000702           1   
695596     251955  E_38954ce11c5ed4  E_7bddda320c3f46  0.001701           1   
965467     791556  E_b1c831da0998fc  E_8e670ed8809695  0.000158           1   
10240446  1121058  E_fc05425b62dae2  E_645f16ebdc8e65  0.002762          17   
1442732    606964  E_886cb083b25b88  E_2e1ae3b64a20c3  0.000300           2   
1536437    794695  E_b27bcc5ce49ba8  E_90fbe167e30fbb  0.000641           2   
6890301    114838  E_19bd50fe041d85  E_8226cab9790ce5  0.006688          12   
4246558    520827  E_750d3e9bcd56f8  E_38827a1f70b5db  0.021435           7   
2281283      7327  E_01ab93c6fa3e29  E_2969dc57689b34  0.004366           4   
8380746    817585  E_b79b4aa5d243fb  E_f723a80f1bf187  0.001682          14   
1548367    818484  E_b7cfadc2722a72  E_1130153980fe6c  0.000462           2   
1029948    920763  E_ced44660dea617  E_723c9ff72f0476  0.000149           1   
2548428    541240  E_799e298930c931  E_ab9017f2ce62f6  0.000355           4   
791901     444539  E_63e9229ff2580d  E_35e2234f9eb388  0.000196           1   
1373098    467898  E_6924c3b97231f8  E_62e29e45d5ff50  0.000065           2   
1558025    837908  E_bc2aa20cad1243  E_9db07250c8f1c9  0.001198           2   
1771111    125673  E_1c2953455fff0d  E_2548f74266b478  0.000397           3   

          label  name_sim  name_gesh  name_leven  name_jaro  ...  \
1095162       0  1.000000   0.941176         1.0   0.976471  ...   
4395281       0  0.000000   0.666667         8.0   0.866863  ...   
570468        0  0.512660   0.689655        16.0   0.687229  ...   
739550        0  1.000000   1.000000         0.0   1.000000  ...   
4838301       0  1.000000   1.000000         0.0   1.000000  ...   
1234786       0  0.487135   0.761905        13.0   0.879443  ...   
3894120       0  0.474017   0.549020        19.0   0.591150  ...   
2033713       0  1.000000   0.840000         4.0   0.832571  ...   
937628        0  0.407207   0.666667         9.0   0.827857  ...   
955102        0  0.000000   0.166667         6.0   0.447619  ...   
3056303       0  0.182486   0.535714        20.0   0.647683  ...   
7505892       0  0.252130   0.567164        26.0   0.860707  ...   
1272298       0  0.000000   0.550000        13.0   0.772222  ...   
985705        0  0.000000   0.400000         7.0   0.000000  ...   
695596        0  0.573602   0.771930        11.0   0.898485  ...   
965467        0  0.149585   0.432432        15.0   0.521549  ...   
10240446      0  0.595165   0.722222        10.0   0.913043  ...   
1442732       0  0.819117   0.769231         9.0   0.925000  ...   
1536437       0  0.437624   0.292683        29.0   0.834286  ...   
6890301       0  0.564839   0.470588        20.0   0.594256  ...   
4246558       0  1.000000   1.000000         0.0   1.000000  ...   
2281283       0  0.303382   0.583333        26.0   0.661340  ...  

In [ ]:
for id, match_id in false_positive.sample(min(false_positive.shape[0], 30))[['id', 'match_id']].values:
    display(data[data['id'].isin([id, match_id])])

id      name   latitude  longitude address city state  \
831298  E_baae62b98df8dd  EML 11-A  39.111336  27.173293     NaN  NaN   NaN   
929424  E_d0c832852333e2  AML 11-A  39.111181  27.174327     NaN  NaN   NaN   

        zip country  url phone  \
831298  NaN      TR  NaN   NaN   
929424  NaN      TR  NaN   NaN   

                                               categories point_of_interest  \
831298                                    Student Centers  P_060a61a6e6e077   
929424  College Classrooms, High Schools, Housing Deve...  P_e6a7fccecc53ff   

        POI_count  set  
831298          0  1.0  
929424          0  1.0

id           name   latitude  longitude  \
801593  E_b403fd274bdf82     My Balcony  41.713735  21.767681   
943901  E_d40601b5bf03aa  In My Kitchen  41.712322  21.767220   

                address   city state  zip country  url phone    categories  \
801593  –ö–∏—Ä–æ –å—É—á—É–∫ –±—Ä.4  –í–µ–ª–µ—Å   NaN  NaN      MK  NaN   NaN           NaN   
943901              NaN  –í–µ–ª–µ—Å   NaN  NaN      MK  NaN   NaN  Salad Places   

       point_of_interest  POI_count  set  
801593  P_1c053b1da7a1f1          0  1.0  
943901  P_6966b5333895c9          0  1.0

id                                name  latitude  \
2916    E_00aeccbc928ac6  Gereja St. Arnoldus Janssen Bekasi -6.249191   
304569  E_4465de54fb257a         Gereja St. Arnoldus Janssen -6.250698   

         longitude address    city      state  zip country  url phone  \
2916    107.006960     NaN     NaN        NaN  NaN      ID  NaN   NaN   
304569  107.011993     NaN  Bekasi  West Java  NaN      ID  NaN   NaN   

       categories point_of_interest  POI_count  set  
2916     Churches  P_57b58ed23ecfc6          0  1.0  
304569   Churches  P_78751d525ed419          2  1.0

id                 name  latitude   longitude  \
175384  E_2755032dc7d86e     Flemington Hotel  4.847474  100.750037   
371471  E_53796fa4921368  Flemington Lounge Óå®  4.847769  100.750174   

                      address     city  state    zip country  url phone  \
175384  1 Jalan Samanea Saman  Taiping  Perak  34000      MY  NaN   NaN   
371471                    NaN      NaN  Perak    NaN      MY  NaN   NaN   

              categories point_of_interest  POI_count  set  
175384            Hotels  P_27f10d1a4c9d73          1  1.0  
371471  Bed & Breakfasts  P_3fb4d865c3d211          0  1.0

id                                       name  \
742099  E_a6b92b38b4ea4c            Ospedale Sant'orsola pad 17 CUP   
793052  E_b21fa7391a030c  Ospedale Policlinico Sant'Orsola-Malpighi   

         latitude  longitude                     address     city state  \
742099  44.491680  11.359724            via massarenti 9  bologna   NaN   
793052  44.491738  11.360059  Via Giuseppe Massarenti, 9  Bologna    BO   

          zip country  url        phone                  categories  \
742099    NaN      IT  NaN          NaN                   Hospitals   
793052  40138      IT  NaN  05163631111  Hospitals, Medical Centers   

       point_of_interest  POI_count  set  
742099  P_585a85dbdb55ae          1  1.0  
793052  P_5968a5c07d381f          0  1.0

...

id          name   latitude  longitude  \
434237  E_619cd893575932  Big 10 Tires  30.417630 -87.275985   
637355  E_8f49aeeeef1499  Pepboys Auto  30.417559 -87.276505   

                        address       city state    zip country  url  \
434237  549 N New Warrington Rd  Pensacola    FL  32506      US  NaN   
637355  549 N New Warrington Rd  Pensacola    FL  32506      US  NaN   

             phone        categories point_of_interest  POI_count  set  
434237  8504551305  Automotive Shops  P_c2f43a8f406edf          1  1.0  
637355         NaN  Automotive Shops  P_be46a9e215d3b7          1  1.0

id                   name   latitude   longitude  \
443783   E_63bd0924afea6a  Emperor Guan‚Äôs Shrine  35.442246  139.645331   
1064340  E_ef34cf3f0bbe38                    Ëê¨ÂíåÊ®ì  35.442277  139.645257   

          address               city     state       zip country  url  \
443783        NaN  Naka-Ku, Yokohama  Kanagawa  231-0023      JP  NaN   
1064340  ‰∏≠Âå∫Â±±‰∏ãÁî∫139                Ê®™ÊµúÂ∏Ç      Á•ûÂ•àÂ∑ùÁúå  231-0023      JP  NaN   

             phone                                  categories  \
443783         NaN                                         NaN   
1064340  456633113  Taiwanese Restaurants, Chinese Restaurants   

        point_of_interest  POI_count  set  
443783   P_99720fb08f3996          1  1.0  
1064340  P_8297eeea2c4f10          1  1.0

id                                        name  \
185155  E_298a661dbf986c  Makati Medical Center Radiology Department   
632616  E_8e3c9ec9165369                       makati medical center   

         latitude   longitude  \
185155  14.559235  121.014720   
632616  14.559796  121.013733   

                                                  address         city  \
185155  GF Tower 1, Makati Medical Center, 2 Amorsolo ...  Makati City   
632616                                                NaN          NaN   

               state   zip country  url phone  \
185155  Metro Manila  1229      PH  NaN   NaN   
632616           NaN   NaN      PH  NaN   NaN   

                                      categories point_of_interest  POI_count  \
185155  Medical Labs, Hospitals, Medical Centers  P_4d4ff12cf3bb39          1   
632616                              Coffee Shops  P_b5433bf426886a          4   

        set  
185155  1.0  
632616  1.0

id           name   latitude  longitude address  \
861116   E_c16aa5a0e739f4  Ds Swiss Food  40.624641 -73.965324     NaN   
1111002  E_f9ba414043228b     Ds Turkish  40.624649 -73.965378     NaN   

             city state  zip country  url phone           categories  \
861116   Brooklyn    NY  NaN      US  NaN   NaN    Swiss Restaurants   
1111002  Brooklyn    NY  NaN      US  NaN   NaN  Turkish Restaurants   

        point_of_interest  POI_count  set  
861116   P_7a529359fe9498          0  1.0  
1111002  P_f30b26a261006a          0  1.0

id                  name   latitude   longitude address  \
145050  E_20847987a3b6ae   WWDC Beer Bash Line  37.785614 -122.401262     NaN   
589092  E_846843374bb8be  WWWDC Beer Bash Line  37.785710 -122.401256     NaN   

                 city state    zip country  url phone         categories  \
145050  San Francisco    CA  94103      US  NaN   NaN  Sculpture Gardens   
589092  San Francisco    CA  94105      US  NaN   NaN  Sculpture Gardens   

       point_of_interest  POI_count  set  
145050  P_07373dd93f20d6          0  1.0  
589092  P_c863f31d052f0d          0  1.0

label„Åå1„Åß1„Å®‰∫àÊ∏¨„Åó„Å¶„ÅÑ„Çã„ÇÇ„ÅÆ

In [ ]:
true_positive = test_data.loc[(test_data["label"] == 1) & (test_data["pred"] > threshold), :]
true_positive.describe()

index          kdist     kneighbors     label       name_sim  \
count  8.631280e+05  863128.000000  863128.000000  863128.0  863128.000000   
mean   5.688986e+05       0.001584       1.069137       1.0       0.888580   
std    3.286761e+05       0.019942       2.723754       0.0       0.256248   
min    0.000000e+00       0.000000       0.000000       1.0       0.000000   
25%    2.843520e+05       0.000000       0.000000       1.0       1.000000   
50%    5.685460e+05       0.000000       0.000000       1.0       1.000000   
75%    8.537002e+05       0.000221       1.000000       1.0       1.000000   
max    1.138811e+06       4.246996      19.000000       1.0       1.000000   

           name_gesh     name_leven      name_jaro       name_lcs  \
count  863127.000000  863127.000000  863127.000000  863127.000000   
mean        0.909067       2.778827       0.941280      15.896033   
std         0.198621       6.510660       0.149922       9.312137   
min         0.000000       0.000000       0.000000       0.000000   
25%         1.000000       0.000000       1.000000       9.000000   
50%         1.000000       0.000000       1.000000      14.000000   
75%         1.000000       0.000000       1.000000      21.000000   
max         1.000000      59.000000       1.000000      64.000000   

           name_CE_1  ...  categories_leven  categories_jaro  categories_lcs  \
count  863127.000000  ...     768090.000000    768090.000000   768090.000000   
mean        0.000073  ...          2.852491         0.947329       16.404709   
std         0.000433  ...          7.817405         0.143676       10.890143   
min         0.000002  ...          0.000000         0.000000        0.000000   
25%         0.000002  ...          0.000000         1.000000        9.000000   
50%         0.000002  ...          0.000000         1.000000       14.000000   
75%         0.000004  ...          0.000000         1.000000       19.000000   
max         0.004961  ...         87.000000         1.000000      122.000000   

       categories_CE_1  categories_CE_2  categories_len_diff  \
count    786755.000000    786721.000000        863128.000000   
mean          0.004602         0.004604             2.828052   
std           0.005588         0.005591             7.762266   
min           0.000002         0.000002             0.000000   
25%           0.000443         0.000443             0.000000   
50%           0.002432         0.002432             0.000000   
75%           0.006273         0.006273             0.000000   
max           0.022032         0.022032            91.000000   

       categories_nleven  categories_nlcsk  categories_nlcs           pred  
count      768090.000000     768090.000000    768090.000000  863128.000000  
mean            0.097295          0.938544         0.938556       0.973729  
std             0.239034          0.188379         0.188392       0.073949  
min             0.000000          0.000000         0.000000       0.563971  
25%             0.000000          1.000000         1.000000       0.996797  
50%             0.000000          1.000000         1.000000       0.999998  
75%             0.000000          1.000000         1.000000       1.000000  
max             1.000000          1.000000         1.000000       1.000000  

[8 rows x 65 columns]

In [ ]:
true_positive.sample(min(true_positive.shape[0], 30))

index                id          match_id     kdist  kneighbors  \
498743    997447  E_e0126da1f381b9  E_e0126da1f381b9  0.000000           0   
312014    623145  E_8c1a47c1b13cc5  E_8c1a47c1b13cc5  0.000000           0   
1555718   833259  E_bb1fbbf0c89acc  E_9ede964c0dc29d  0.001079           2   
63144     126175  E_1c46b45de5d4bc  E_1c46b45de5d4bc  0.000000           0   
704094    269050  E_3c6bc889d750bc  E_1aa0db378965a4  0.000869           1   
10369      20697  E_04a5ddb34ea3eb  E_04a5ddb34ea3eb  0.000000           0   
674073    209027  E_2ee90af5d13c04  E_2cfb6a914c1a08  0.000134           1   
460667    920994  E_cee2abfb2d6c51  E_cee2abfb2d6c51  0.000000           0   
369761    738908  E_a5fdd09804cb19  E_a5fdd09804cb19  0.000000           0   
167149    333925  E_4b0d2dbfcb19e6  E_4b0d2dbfcb19e6  0.000000           0   
1039594   940043  E_d32557bd433aba  E_fb925a24c3aa29  0.009068           1   
750524    361695  E_51476a26070d5f  E_833bb746a19d94  0.000151           1   
1183908    90261  E_143a91f4b9af6c  E_c36332ae505e1b  0.000322           2   
1024021   908977  E_cc3549405207da  E_79414095d3b7fc  0.000258           1   
263284    525997  E_76324f19ef6555  E_76324f19ef6555  0.000000           0   
550518   1100897  E_f770cb0b5bd3cc  E_f770cb0b5bd3cc  0.000000           0   
1354150   430487  E_60bf53eb18e265  E_ff0927cfde45fc  0.000299           2   
495969    991865  E_ded007bd9df390  E_ded007bd9df390  0.000000           0   
71029     142181  E_1fde2297c6278a  E_1fde2297c6278a  0.000000           0   
171335    342323  E_4cedd4e50332a4  E_4cedd4e50332a4  0.000000           0   
848748    558341  E_7d7c6705291897  E_fe5548d314a490  0.000494           1   
527398   1054481  E_ecf2269a635b92  E_ecf2269a635b92  0.000000           0   
6858541    51335  E_0b7e1bcf877a07  E_b2e0c06fc922f5  0.005950          12   
890752    641887  E_9048df1a8822c2  E_5ff9d881fae1dc  0.000287           1   
331675    662614  E_94e14d90b4a78a  E_94e14d90b4a78a  0.000000           0   
4593605    76859  E_1131d1d37e5d38  E_b3b826551f247a  0.001539           8   
1100734  1062285  E_eebef2abb843f8  E_1e96ed20de734e  0.000550           1   
383992    767397  E_ac601ce9743ac4  E_ac601ce9743ac4  0.000000           0   
485351    970541  E_da00d107f27013  E_da00d107f27013  0.000000           0   
1145018    12363  E_02ce4f71320b9a  E_9125bfd9ad629c  0.000306           2   

         label  name_sim  name_gesh  name_leven  name_jaro  ...  \
498743       1  1.000000   1.000000         0.0   1.000000  ...   
312014       1  1.000000   1.000000         0.0   1.000000  ...   
1555718      1  1.000000   1.000000         0.0   1.000000  ...   
63144        1  1.000000   1.000000         0.0   1.000000  ...   
704094       1  1.000000   1.000000         0.0   1.000000  ...   
10369        1  1.000000   1.000000         0.0   1.000000  ...   
674073       1  0.579870   0.454545        20.0   0.622757  ...   
460667       1  1.000000   1.000000         0.0   1.000000  ...   
369761       1  1.000000   1.000000         0.0   1.000000  ...   
167149       1  1.000000   1.000000         0.0   1.000000  ...   
1039594      1  1.000000   1.000000         0.0   1.000000  ...   
750524       1  0.617887   0.487805        21.0   0.864516  ...   
1183908      1  1.000000   0.166667         5.0   0.444444  ...   
1024021      1  0.575860   0.562500        13.0   0.815758  ...   
263284       1  1.000000   1.000000         0.0   1.000000  ...   
550518       1  1.000000   1.000000         0.0   1.000000  ...   
1354150      1  0.751661   0.588235         7.0   0.883333  ...   
495969       1  1.000000   1.000000         0.0   1.000000  ...   
71029        1  1.000000   1.000000         0.0   1.000000  ...   
171335       1  1.000000   1.000000         0.0   1.000000  ...   
848748       1  0.000000   0.114286        25.0   0.384259  ...   
527398       1  1.000000   1.000000         0.0   1.000000  ...   
6858541      1  0.438600   0.928571         1.0   0

In [ ]:
for id, match_id in true_positive.sample(min(true_positive.shape[0], 30))[['id', 'match_id']].values:
    display(data[data['id'].isin([id, match_id])])

id                       name   latitude   longitude  \
559692  E_7dcbf874cc792b  The Ritz-Carlton, Chengdu  30.660192  104.070025   
646328  E_914501442bc955  The Ritz-Carlton, Chengdu  30.668222  104.068508   

                                        address     city    state     zip  \
559692  269 Shuncheng Avenue, Qingyang District      ÊàêÈÉΩÂ∏Ç    CN-51  610017   
646328  269 Shuncheng Avenue, Qingyang District  Chengdu  Sichuan  610017   

       country                                                url  \
559692      CN  http://www.ritzcarlton.com/en/hotels/china/che...   
646328      CN  https://www.ritzcarlton.com/en/hotels/china/ch...   

                phone                      categories point_of_interest  \
559692  +862883588888  Hotels, Resorts, Meeting Rooms  P_74efbfa6bf8967   
646328  +862883588888                          Hotels  P_74efbfa6bf8967   

        POI_count  set  
559692          1  1.0  
646328          1  1.0

id       name  latitude  longitude address city state  \
317035  E_473e73e586fefa  Casa Rios -5.084937 -42.790379     NaN  NaN   NaN   

        zip country  url phone         categories point_of_interest  \
317035  NaN      BR  NaN   NaN  Shoe Repair Shops  P_2dcd6773846fec   

        POI_count  set  
317035          1  1.0

id    name   latitude   longitude  address city state  \
877241  E_c50c792af282d3  ÈßÖ„Åù„Å∞ ÁàΩ‰∫≠  35.714385  139.777322  ‰∏äÈáé7-1-1  Âè∞Êù±Âå∫   Êù±‰∫¨ÈÉΩ   

             zip country              url phone        categories  \
877241  110-0005      JP  http://jt-s.net   NaN  Soba Restaurants   

       point_of_interest  POI_count  set  
877241  P_c57975c175abd3          2  1.0

id                         name   latitude  longitude  \
527795  E_769954deb707f5  Igreja Nossa Senhora da Luz -13.005099  -38.45672   

       address      city  state  zip country  url phone categories  \
527795     NaN  Salvador  Bahia  NaN      BR  NaN   NaN   Churches   

       point_of_interest  POI_count  set  
527795  P_aad612004c2254          0  1.0

id                          name  latitude  longitude  \
591091  E_84dc4a9cbd1a9d  Kahriman Nakliyat Emlak Spot  38.01531  32.518365   

                                             address      city   state    zip  \
591091  Bosna Hersek Mah. Soner Sk. Ba≈ü √áar≈üƒ± No:1/Z  Sel√ßuklu  Turkey  42250   

       country                             url        phone  \
591091      TR  http://kahrimannakliyat.com.tr  05446819413   

                                          categories point_of_interest  \
591091  Real Estate Offices, Thrift / Vintage Stores  P_2c82501b178e91   

        POI_count  set  
591091          4  1.0

...

id    name   latitude  longitude address    city  \
127096  E_1c7c2d96c2022d  Jax 01  24.745183  46.537605     NaN  Riyadh   

         state    zip country  url phone                  categories  \
127096  Riyadh  13732      SA  NaN   NaN  Art Studios, Art Galleries   

       point_of_interest  POI_count  set  
127096  P_ad13759f563318          1  1.0

id                 name   latitude   longitude address  \
991843  E_decefff1250549  ‡πÄ‡∏Æ‡∏∑‡∏≠‡∏ô‡∏™‡∏ß‡∏ô‡∏ö‡πâ‡∏≤‡∏ô‡∏î‡∏≠‡∏ô‡∏ò‡∏£‡∏£‡∏°  16.409487  103.584128     NaN   

       city state  zip country  url phone        categories point_of_interest  \
991843  NaN   NaN  NaN      TH  NaN   NaN  Thai Restaurants  P_acd5ae0b4d9bda   

        POI_count  set  
991843          1  1.0

id                                  name  latitude  \
162587  E_2472d6a80ea912                      Shangri-La Hotel -7.290713   
674373  E_977d24fe0e8643  Shangri-La hotel Surabaya, Indonesia -7.290732   

         longitude                      address      city       state    zip  \
162587  112.716734     Mayjen Sungkono Building       NaN         NaN    NaN   
674373  112.715966  Jl. May. Jend. Sungkono 120  Surabaya  Jawa Timur  60256   

       country  url      phone categories point_of_interest  POI_count  set  
162587      ID  NaN        NaN        NaN  P_a64aea59108a28          4  1.0  
674373      ID  NaN  315661570     Hotels  P_a64aea59108a28          4  1.0

id                    name   latitude  longitude  \
133492  E_1deb11d034a3da  Condom√≠nio Ecovillagio -23.512372 -47.125592   
519732  E_74cf01f08ec50e            Ecovillaggio -23.512412 -47.125527   

                        address       city      state        zip country  url  \
133492  Estrada Da Serrinha 291  S√£o Roque         SP  18133-310      BR  NaN   
519732      Estrada da Serrinha  S√£o Roque  S√£o Paulo        NaN      BR  NaN   

       phone                                         categories  \
133492   NaN        Residential Buildings (Apartments / Condos)   
519732   NaN  Housing Developments, Residential Buildings (A...   

       point_of_interest  POI_count  set  
133492  P_d3fdd4cbf33af8          1  1.0  
519732  P_d3fdd4cbf33af8          1  1.0

id                       name   latitude  longitude  \
465555  E_68989766229cb4  GARAGE DES PRAZ - PEUGEOT  45.940002   6.881045   

                    address                 city        state    zip country  \
465555  1273 ROUTE DES PRAZ  Chamonix-Mont-Blanc  Rh√¥ne-Alpes  74400      FR   

                                                 url         phone  \
465555  http://agents.peugeot.fr/chamonix-mont-blanc  +33450531432   

          categories point_of_interest  POI_count  set  
465555  Auto Garages  P_c128200e0930e5          0  1.0

label„Åå0„Åß0„Å®‰∫àÊ∏¨„Åó„Å¶„ÅÑ„Çã„ÇÇ„ÅÆ

In [ ]:
true_negative = test_data.loc[(test_data["label"] == 0) & (test_data["pred"] <= threshold), :]
true_negative.describe()

In [ ]:
true_negative.sample(min(true_negative.shape[0], 30))

index                id          match_id     kdist  kneighbors  \
8091808    239909  E_35da0a92640ab1  E_77acb95fdc5812  0.046287          14   
8623906    165570  E_251e27f1563d05  E_704058088575d6  0.274183          15   
10510546   521946  E_754f52edd3a379  E_a36d2b588b56ef  0.009249          18   
10143907   927738  E_d062fe6c8e42c6  E_43ba2b22052bb6  0.095637          17   
10603397   707474  E_9ee75a52f4f52a  E_ad2efde731224f  0.007662          18   
9044114   1005991  E_e1fa942b947e24  E_8d991b29b79b00  0.032109          15   
10139395   918771  E_ce60d023e47e87  E_0c6266bcd3feb6  0.053493          17   
1578482    878932  E_c56b888fc550d5  E_7729ab44fe0713  0.000679           2   
875007     610357  E_89385f2ba1662a  E_c16ef99eb77942  0.001111           1   
9472655    723695  E_a28e4742077a3e  E_aba8d1ed08cbd3  0.001376          16   
7211025    755723  E_a9c4b2be73bd98  E_30fe3cd53bc8df  0.093300          12   
715844     292605  E_41b52253bd345e  E_1c1c66dcff02c4  0.000821           1   
891179     642729  E_907abff5e301ed  E_bad7f585d879f1  0.001889           1   
1001700    864054  E_c215ac92c7d219  E_fb211fec20178d  0.008624           1   
6697085    866808  E_c2b52a4675d5de  E_23484654699a7f  0.105588          11   
8440772    937872  E_d2ab1c69f26c75  E_f3773f09c06697  0.027473          14   
6038731    688704  E_9aad674365d0fa  E_2e7aaf70a0814a  0.004621          10   
8715905    349356  E_4e827a0864333c  E_39a94728ace871  0.003283          15   
7098177    530203  E_7725dc1bc203d8  E_e2117fb9bf4113  0.004282          12   
3391378   1088581  E_f49daec0aed958  E_d095eff40ee9b4  0.023271           5   
5249268    248941  E_37e023436f09bf  E_79ae884eb3aa86  0.021927           9   
10194079  1028049  E_e6fa2176450b95  E_64bcc823f8c104  0.071522          17   
7665385    525697  E_76211f912ba8f9  E_5e623c2d193bdd  0.002694          13   
4174022    375848  E_5473f5e9af0a92  E_29be3bdb4bc891  0.012226           7   
10414350   329730  E_4a1dc2fe6d90ed  E_21bf77b32a99e3  0.005249          18   
5307369    364900  E_51feb63176ead9  E_7e7c1de14afe22  0.003162           9   
1601865    925770  E_cff1cc786ebb61  E_ab61e287b45309  0.003918           2   
11074723   511515  E_72fc36c1501a3b  E_7145592ca01803  0.010782          19   
702056     264987  E_3b83588e93989a  E_486034797602c5  0.010850           1   
1780618    144890  E_207abac4942fb7  E_89585b45404cab  0.000254           3   

          label  name_sim  name_gesh  name_leven  name_jaro  ...  \
8091808       0  0.000000   0.097561        25.0   0.330688  ...   
8623906       0  0.000000   0.133333        16.0   0.357969  ...   
10510546      0  0.000000   0.055556        27.0   0.273810  ...   
10143907      0  0.000000   0.080000        15.0   0.388889  ...   
10603397      0  0.000000   0.000000        11.0   0.000000  ...   
9044114       0  0.000000   0.200000        35.0   0.479242  ...   
10139395      0  0.000000   0.200000         9.0   0.468013  ...   
1578482       0  0.000000   0.000000         5.0   0.000000  ...   
875007        0  0.000000   0.083333        15.0   0.392593  ...   
9472655       0  0.000000   0.178571        30.0   0.505556  ...   
7211025       0  0.000000   0.080000        19.0   0.416667  ...   
715844        0  0.000000   0.160000        14.0   0.449074  ...   
891179        0  0.798245   0.866667         3.0   0.948214  ...   
1001700       0  0.000000   0.275862        14.0   0.459596  ...   
6697085       0  0.000000   0.333333        26.0   0.611111  ...   
8440772       0  0.000000   0.294118        14.0   0.465939  ...   
6038731       0  0.000000   0.000000        18.0   0.000000  ...   
8715905       0  0.000000   0.095238        15.0   0.420833  ...   
7098177       0  0.000000   0.173913        15.0   0.470238  ...   
3391378       0  0.000000   0.117647        10.0   0.419192  ...   
5249268       0  0.000000   0.200000        18.0   0.422723  ...   
10194079      0  0.000000   0.000000         9.0   0.000000  ...  

In [ ]:
for id, match_id in true_negative.sample(min(true_negative.shape[0], 30))[['id', 'match_id']].values:
    display(data[data['id'].isin([id, match_id])])

id                name  latitude   longitude  \
793876   E_b24c624904ee22      Menara Citicon -6.192571  106.797832   
1004064  E_e18c3df30dce99  Akamsi 24 Building -6.194441  106.797573   

                                  address           city        state    zip  \
793876   Jl. Let. Jend. S. Parman Kav. 72  Jakarta Barat  DKI Jakarta  11410   
1004064                               NaN            NaN          NaN    NaN   

        country                       url       phone  \
793876       ID  http://menaraciticon.com  2153670077   
1004064      ID                       NaN         NaN   

                          categories point_of_interest  POI_count  set  
793876   Buildings, Offices, Mosques  P_420efa23b17706          1  1.0  
1004064                          NaN  P_48189c115f4bd1          1  1.0

id                             name   latitude  \
443868  E_63c1cee6396917  Swedish Snoqualmie Primary Care  47.533922   
516508  E_741756a7ac7a9c                        VF Outlet  47.490418   

         longitude            address        city state    zip country  \
443868 -121.839066   37624 SE Fury St  Snoqualmie    WA  98065      US   
516508 -121.794924  521 S Fork Ave SW  North Bend    WA  98045      US   

                                                      url       phone  \
443868  https://www.swedish.org/locations/swedish-snoq...  4258882016   
516508                            http://www.vfoutlet.com  4258311700   

                              categories point_of_interest  POI_count  set  
443868                  Doctor's Offices  P_61e9e5fa7afadd          1  1.0  
516508  Clothing Stores, Lingerie Stores  P_601ad45aa03d37          0  1.0

id          name   latitude   longitude         address  \
414422  E_5d1be87310eb20  Í¥ëÏÇ∞Íµ¨ Í≥µÏùµÌôúÎèôÏßÄÏõêÏÑºÌÑ∞  35.195090  126.822381  Í¥ëÏÇ∞Íµ¨ Ïû•ÎçïÎ°ú96Î≤àÍ∏∏ 15   
885320  E_c6d7617d20f249  Caf√© DROPTOP  35.153772  126.850243  ÏÑúÍµ¨ ÏÉÅÎ¨¥Ï§ëÏïôÎ°ú78Î≤àÍ∏∏ 9   

           city    state    zip country                         url  \
414422    Í¥ëÏ£ºÍ¥ëÏó≠Ïãú      NaN  62224      KR  http://www.maeulings.or.kr   
885320  Gwangju  Gwangju    NaN      KR                         NaN   

            phone                                   categories  \
414422  629598526  Government Buildings, General Entertainment   
885320  623817877                                 Coffee Shops   

       point_of_interest  POI_count  set  
414422  P_ae11d8736596c9          1  1.0  
885320  P_a2aa9fa38f6c85          1  1.0

id                   name   latitude  longitude  \
552910  E_7c4686c09709d0  Woodlands Office Park -26.061458  28.087895   
725871  E_a30c436a4954d5          College house -26.043901  28.059876   

              address       city    state  zip country  url phone categories  \
552910  Woodlands Dr.  Woodlands  Gauteng  NaN      ZA  NaN   NaN    Offices   
725871    Gary street   Pretoria  Gauteng  NaN      ZA  NaN   NaN        NaN   

       point_of_interest  POI_count  set  
552910  P_c526e5b477c1a1          0  1.0  
725871  P_003c8bfa6a9005          1  1.0

id                       name   latitude  longitude  \
122603  E_1b7ddb964a5d49  Leeds Restaurant & Lounge  40.243961 -76.808381   
146858  E_20ed96ffe4aad4          Hilton Garden Inn  40.251314 -76.817973   

                    address        city state    zip country  \
122603  750 Eisenhower Blvd  Harrisburg    PA  17111      US   
146858      3943 Tecport Dr  Harrisburg    PA  17111      US   

                                                      url       phone  \
122603            http://www.leedsrestaurantandlounge.com  7175644654   
146858  https://www.hilton.com/en/hotels/mdthagi-hilto...  7176357299   

                        categories point_of_interest  POI_count  set  
122603  American Restaurants, Bars  P_4058f78a38a86b          1  1.0  
146858                      Hotels  P_262e45551a271c          1  1.0

...

id                  name   latitude  longitude  \
484724  E_6cf109af1f4bf4  Pop Century Bus Stop  28.350260 -81.542558   
705875  E_9e8bc302180b58              Barbados  28.360303 -81.542081   

                       address              city state    zip country  url  \
484724         1050 Century Dr  Lake Buena Vista    FL  32830      US  NaN   
705875  Caribbean Beach Resort  Lake Buena Vista    FL  32830      US  NaN   

             phone categories point_of_interest  POI_count  set  
484724  4079384000  Bus Stops  P_69e0c762355b69          0  1.0  
705875  4079343400     Hotels  P_88600ded95e3c8          0  1.0

id             name   latitude  longitude      address  \
480128  E_6be5407f96cd89  Milliken Europe  51.058135   3.733306  Ham 18 - 24   
557938  E_7d63f77fc11365         Baravins  51.057129   3.731171  Steendam 53   

        city            state   zip country                     url  \
480128  Gent  Oost-Vlaanderen  9000      BE                     NaN   
557938  Gent              NaN  9000      BE  http://www.baravins.be   

              phone categories point_of_interest  POI_count  set  
480128          NaN  Factories  P_380eda9d306fff          1  1.0  
557938  +3293240085  Wine Bars  P_8e36ed229ba360          1  1.0

id                       name  latitude   longitude  \
558877  E_7d9d30d4be2161  risma seafood pangandaran -7.391682  108.540462   
776220  E_ae5a841352ea1b                 BJB Banjar -7.372450  108.544511   

       address city state  zip country  url phone categories  \
558877     NaN  NaN   NaN  NaN      ID  NaN   NaN        NaN   
776220     NaN  NaN   NaN  NaN      ID  NaN   NaN      Banks   

       point_of_interest  POI_count  set  
558877  P_4b649379cf5ce0          3  1.0  
776220  P_18c3491c9778d2          1  1.0

id       name   latitude   longitude address  city  \
185248  E_298ff2ebdc479f     „Åµ„Çå„ÅÇ„ÅÑ„ÅÆÊ£Æ  34.726543  135.675960     NaN  ÂõõÊ¢ùÁï∑Â∏Ç   
681759  E_9926f86675a920  „Åø„Åö„ÅªÈäÄË°åÁîüÈßíÊîØÂ∫ó  34.693670  135.696648     NaN   NaN   

       state       zip country  url phone categories point_of_interest  \
185248   Â§ßÈò™Â∫ú  575-0011      JP  NaN   NaN    Forests  P_07185ffef7a353   
681759   NaN       NaN      JP  NaN   NaN      Banks  P_7b04c97d548cac   

        POI_count  set  
185248          0  1.0  
681759          1  1.0

id                                           name  \
902289   E_caa97f75858c8c                  Bengkel Plat Nomor Sekeluarga   
1029892  E_e7624ac5948953  Business Suite [Arion Swiss-Belhotel Bandung]   

         latitude   longitude                          address     city  \
902289  -6.910300  107.604008                   Jl. Cicendo 29  Bandung   
1029892 -6.913337  107.604325  Jalan Otto Iskandardinata No 16  Bandung   

              state  zip country  url phone        categories  \
902289   Jawa Barat  NaN      ID  NaN   NaN  Automotive Shops   
1029892   West Java  NaN      ID  NaN   NaN            Hotels   

        point_of_interest  POI_count  set  
902289   P_a8c4455b248b4f          0  1.0  
1029892  P_9274027826409c          4  1.0